In [1]:
!pip install bs4 pandas requests

In [2]:
#Import packages here
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
#Import MicroBigge file here
try:
    df = pd.read_csv('/content/microbigge.csv')
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")
    # Inspect the problematic row
    !head -n 182200 '/content/microbigge.csv' | tail -n 5
    # Consider adding the 'error_bad_lines=False' argument to skip the problematic row
    # df = pd.read_csv('/content/microbigge.csv', error_bad_lines=False)

In [ ]:
df = pd.read_csv('/content/microbigge.csv', on_bad_lines='skip')



ParserError: Error tokenizing data. C error: EOF inside string starting at row 182199

In [3]:
df = pd.read_csv('/content/microbigge_ecoli.csv')

In [4]:
df.shape

(23295, 18)

In [5]:
df['Type'].value_counts(normalize=True)

Type
AMR          0.504958
VIRULENCE    0.426830
STRESS       0.068212
Name: proportion, dtype: float64

In [6]:
df2 = df[df['#Scientific name'] == 'Escherichia coli']

In [7]:
df2['Type'].value_counts(normalize=True)

Type
AMR          0.504958
VIRULENCE    0.426830
STRESS       0.068212
Name: proportion, dtype: float64

In [8]:
df2.to_csv('microbigge_ecoli.csv', index=False)

In [6]:
# Define a function to fetch the DNA sequence from the NCBI website
def fetch_dna_sequence(contig, start, stop):
    url = f'https://www.ncbi.nlm.nih.gov/nuccore/{contig}?from={start}&to={stop}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

In [7]:
# Define a function to extract the nucleotide sequence from the ORIGIN section
def extract_origin_sequence(data):
    if not data:
        return None
    origin_start = data.find("ORIGIN")
    if origin_start == -1:
        return None
    origin_section = data[origin_start:]
    sequence = re.sub(r'[^atcgn]', '', origin_section.lower())
    return sequence

In [8]:
# Apply the functions to each row in the DataFrame and add a new column for the DNA sequence
def get_dna_sequence(row):
    data = fetch_dna_sequence(row['Contig'], row['Start'], row['Stop'])
    sequence = extract_origin_sequence(data)
    return sequence

In [ ]:
df['DNA_Sequence'] = df.apply(get_dna_sequence, axis=1)

# Save the updated DataFrame back to the CSV
df.to_csv('fullseq_microbigge_ecoli.csv', index=False)